In [6]:
import os
from collections import OrderedDict
from io import StringIO

import anndata
import numpy as np
import pandas as pd
import scanpy

scanpy.settings.verbosity = 3
scanpy.logging.print_versions()

pd.options.display.max_columns = 100

outdir_gdrive = '/home/olga/googledrive/TabulaMicrocebus/data/cross-species'
outdir_local = '/home/olga/data_lg/data_sm_copy/tabula-microcebus/data-objects/cross-species'
! mkdir $outdir_local $outdir_gdrive
outdirs = outdir_gdrive, outdir_local
# for d in outdirs:
#     ! ls -lha $d

scanpy==1.4.6.dev9+ge632939 anndata==0.6.22.post1 umap==0.4.6 numpy==1.17.3 scipy==1.3.2 pandas==0.25.3 scikit-learn==0.22 statsmodels==0.10.2 python-igraph==0.7.1 louvain==0.6.1
mkdir: cannot create directory '/home/olga/data_lg/data_sm_copy/tabula-microcebus/data-objects/cross-species': File exists
mkdir: cannot create directory '/home/olga/googledrive/TabulaMicrocebus/data/cross-species': File exists


# Read in orthologous genes

In [7]:
orthologous_genes_folder = '/home/olga/googledrive/TabulaMicrocebus/data/orthologous-genes'
# ! ls -lha $orthologous_genes_folder

In [8]:
# ! ls -lha $orthologous_genes_folder/Ensembl_NCBI_MGI_homology

In [9]:
# ! ls -lha $orthologous_genes_folder/Ensembl_NCBI_MGI_homology/NCBI_lemur_homologs_summary_one2one.csv

## Use cleaned orthologies from Camille

In [10]:
gene_orthologies = pd.read_csv(f"{orthologous_genes_folder}/Ensembl_NCBI_MGI_homology/NCBI_lemur_homologs_summary_one2one.csv", index_col=0)
print(gene_orthologies.shape)
gene_orthologies.head()

(15815, 34)


,NCBI_lemur_geneID,NCBI_lemur_geneName,NCBI_lemur_geneDescription,NCBI_lemur_geneType,Ensembl_human_homology,Ensembl_mouse_homology,NCBI_human_homology,NCBI_mouse_homology,any_mouse_homology,any_human_homology,NCBI_human_geneID_1,NCBI_human_geneName_1,NCBI_mouse_geneID_1,NCBI_mouse_geneName_1,Ensembl_lemur_geneID_1,Ensembl_lemur_geneID_2,Ensembl_lemur_geneName_1,Ensembl_lemur_geneName_2,Ensembl_human_geneID_1,Ensembl_human_geneID_2,Ensembl_human_geneName_1,Ensembl_human_geneName_2,Ensembl_mouse_geneID_1,Ensembl_mouse_geneID_2,Ensembl_mouse_geneName_1,Ensembl_mouse_geneName_2,Ensembl_human_geneID_AGG,Ensembl_human_geneName_AGG,Ensembl_mouse_geneID_AGG,Ensembl_mouse_geneName_AGG,NCBI_human_geneID_AGG,NCBI_human_geneName_AGG,NCBI_mouse_geneID_AGG,NCBI_mouse_geneName_AGG
1,105854774,PDE6D,phosphodiesterase 6D,protein-coding,ortholog_one2one,ortholog_one2one,ortholog,ortholog,yes,yes,5147.0,PDE6D,18582.0,Pde6d,ENSMICG00000032098,NaN,PDE6D,NaN,ENSG00000156973,NaN,PDE6D,NaN,ENSMUSG00000026239,NaN,Pde6d,NaN,ENSG00000156973,PDE6D,ENSMUSG00000026239,Pde6d,5147.0,PDE6D,18582.0,Pde6d
2,105854783,NUDT12,nudix hydrolase 12,protein-coding,ortholog_one2one,ortholog_one2one,ortholog,ortholog,yes,yes,83594.0,NUDT12,67993.0,Nudt12,ENSMICG00000036782,NaN,NUDT12,NaN,ENSG00000112874,NaN,NUDT12,NaN,ENSMUSG00000024228,NaN,Nudt12,NaN,ENSG00000112874,NUDT12,ENSMUSG00000024228,Nudt12,83594.0,NUDT12,67993.0,Nudt12
3,105854790,PI15,peptidase inhibitor 15,protein-coding,ortholog_one2one,ortholog_one2one,ortholog,ortholog,yes,yes,51050.0,PI15,94227.0,Pi15,ENSMICG00000003386,NaN,PI15,NaN,ENSG00000137558,NaN,PI15,NaN,ENSMUSG00000067780,NaN,Pi15,NaN,ENSG00000137558,PI15,ENSMUSG00000067780,Pi15,51050.0,PI15,94227.0,Pi15
4,105854791,GDAP1,ganglioside induced differentiation associated...,protein-coding,ortholog_one2one,ortholog_one2one,ortholog,ortholog,yes,yes,54332.0,GDAP1,14545.0,Gdap1,ENSMICG00000026766,NaN,GDAP1,NaN,ENSG00000104381,NaN,GDAP1,NaN,ENSMUSG00000025777,NaN,Gdap1,NaN,ENSG00000104381,GDAP1,ENSMUSG00000025777,Gdap1,54332.0,GDAP1,14545.0,Gdap1
5,105854792,JPH1,junctophilin 1,protein-coding,ortholog_one2one,ortholog_one2one,ortholog,ortholog,yes,yes,56704.0,JPH1,57339.0,Jph1,ENSMICG00000000667,NaN,JPH1,NaN,ENSG00000104369,NaN,JPH1,NaN,ENSMUSG00000042686,NaN,Jph1,NaN,ENSG00000104369,JPH1,ENSMUSG00000042686,Jph1,56704.0,JPH1,57339.0,Jph1


In [11]:
gene_orthologies_gene_names = gene_orthologies[['NCBI_lemur_geneName', 'NCBI_human_geneName_AGG', 'NCBI_mouse_geneName_AGG']]
print(gene_orthologies_gene_names.shape)
gene_orthologies_gene_names.head()

(15815, 3)


,NCBI_lemur_geneName,NCBI_human_geneName_AGG,NCBI_mouse_geneName_AGG
1,PDE6D,PDE6D,Pde6d
2,NUDT12,NUDT12,Nudt12
3,PI15,PI15,Pi15
4,GDAP1,GDAP1,Gdap1
5,JPH1,JPH1,Jph1


In [12]:
gene_orthologies_gene_names.sample(10)

,NCBI_lemur_geneName,NCBI_human_geneName_AGG,NCBI_mouse_geneName_AGG
5888,PKDCC,PKDCC,Pkdcc
11758,LOC105878286,AICDA,Aicda
4489,AMPD2,AMPD2,Ampd2
10413,RGS19,RGS19,Rgs19
15060,BAHCC1,BAHCC1,Bahcc1
11706,CHRNB3,CHRNB3,Chrnb3
11802,LOC105878375,MYH8,Myh8
11882,ARL2,ARL2,Arl2
35,FAM219B,FAM219B,Fam219b
3421,LIMD2,LIMD2,Limd2


## Filter for only 1:1 orthologs across all three species

In [13]:
gene_orthologies_gene_names_dropna = gene_orthologies_gene_names.dropna()
print(gene_orthologies_gene_names_dropna.shape)
gene_orthologies_gene_names_dropna

(15810, 3)


,NCBI_lemur_geneName,NCBI_human_geneName_AGG,NCBI_mouse_geneName_AGG
1,PDE6D,PDE6D,Pde6d
2,NUDT12,NUDT12,Nudt12
3,PI15,PI15,Pi15
4,GDAP1,GDAP1,Gdap1
5,JPH1,JPH1,Jph1
...,...,...,...
15811,GNG10,GNG10,Gng10
15812,PRRT1B,PRRT1B,Gm13420
15813,TOMM7,TOMM7,Tomm7
15814,LOC109731319,COX7C,Cox7c


In [14]:
gene_orthologies_gene_names_dropna.drop_duplicates()

,NCBI_lemur_geneName,NCBI_human_geneName_AGG,NCBI_mouse_geneName_AGG
1,PDE6D,PDE6D,Pde6d
2,NUDT12,NUDT12,Nudt12
3,PI15,PI15,Pi15
4,GDAP1,GDAP1,Gdap1
5,JPH1,JPH1,Jph1
...,...,...,...
15811,GNG10,GNG10,Gng10
15812,PRRT1B,PRRT1B,Gm13420
15813,TOMM7,TOMM7,Tomm7
15814,LOC109731319,COX7C,Cox7c


### Find genes that appear only once in the column

In [15]:


gene_name_triplets = {}


for col in gene_orthologies_gene_names_dropna:
    print(f'--- {col} ---')
    df = gene_orthologies_gene_names_dropna.groupby(col).filter(lambda x: len(x) == 1)
    print(df.shape)
    gene_name_triplets[col] = set(map(tuple, df.values))

--- NCBI_lemur_geneName ---
(15810, 3)
--- NCBI_human_geneName_AGG ---
(15527, 3)
--- NCBI_mouse_geneName_AGG ---
(15515, 3)


In [16]:
df.head()

,NCBI_lemur_geneName,NCBI_human_geneName_AGG,NCBI_mouse_geneName_AGG
1,PDE6D,PDE6D,Pde6d
2,NUDT12,NUDT12,Nudt12
3,PI15,PI15,Pi15
4,GDAP1,GDAP1,Gdap1
5,JPH1,JPH1,Jph1


### Use human-lemur-mouse gene name triplets that appear in every perspective

Meaning, these genes name triplets were found if you filter for only genes that appear once in every species' column

In [17]:
one2ones_triplets = gene_name_triplets['NCBI_lemur_geneName'] & gene_name_triplets['NCBI_human_geneName_AGG'] & gene_name_triplets['NCBI_mouse_geneName_AGG']
one2ones_df = pd.DataFrame(one2ones_triplets)
print(one2ones_df.shape)
one2ones_df.head()

(15514, 3)


,0,1,2
0,IQCE,IQCE,Iqce
1,C10H9orf50,C9orf50,1700001O22Rik
2,PPP1R14A,PPP1R14A,Ppp1r14a
3,NOX4,NOX4,Nox4
4,RNFT2,RNFT2,Rnft2


## Make the one2one gene orthology dataframe

In [18]:
renamer = {0: 'mouse_lemur__gene_name', 1: 'human__gene_name', 2: 'mouse__gene_name'}
gene_orthologies_one2one_all_three = one2ones_df.rename(columns=renamer)
print(gene_orthologies_one2one_all_three.shape)
gene_orthologies_one2one_all_three.head()

(15514, 3)


,mouse_lemur__gene_name,human__gene_name,mouse__gene_name
0,IQCE,IQCE,Iqce
1,C10H9orf50,C9orf50,1700001O22Rik
2,PPP1R14A,PPP1R14A,Ppp1r14a
3,NOX4,NOX4,Nox4
4,RNFT2,RNFT2,Rnft2


In [19]:
gene_orthologies_one2one_all_three.query('human__gene_name == "ACE2"')

,mouse_lemur__gene_name,human__gene_name,mouse__gene_name
4536,ACE2,ACE2,Ace2


In [20]:
gene_orthologies_one2one_all_three.sample(10, random_state=0)

,mouse_lemur__gene_name,human__gene_name,mouse__gene_name
4462,IER5L,IER5L,Ier5l
7799,KCNA1,KCNA1,Kcna1
9957,SUOX,SUOX,Suox
5274,ARRDC2,ARRDC2,Arrdc2
167,LOC105877524,BEND5,Bend5
2007,MLIP,MLIP,Mlip
1129,ATP6V1E2,ATP6V1E2,Atp6v1e2
1136,ANKDD1B,ANKDD1B,Ankdd1b
610,BMPR2,BMPR2,Bmpr2
10692,FOXN3,FOXN3,Foxn3


## Write to file!!

In [21]:
gene_orthologies_one2one_all_three.to_csv(f"{orthologous_genes_folder}/ncbi_mgi_ensembl__mouse-lemur_human_mouse__orthologs__gene_names__one2one.csv")

for d in outdirs:
    gene_orthologies_one2one_all_three.to_csv(f"{d}/ncbi_mgi_ensembl__mouse-lemur_human_mouse__orthologs__gene_names__one2one.csv")

# Mouse blood (Camille Ezran's dataset)

In [22]:
h5ad = '/home/olga/data_lg/data_sm_copy/tabula-microcebus/data-objects/mouse_blood_ce.h5ad'

ce_mouse_blood = scanpy.read(h5ad, cache=True)

## Subset on 1:1 orthologs

In [18]:
gene_mask = ce_mouse_blood.var.index.isin(
    gene_orthologies_one2one_all_three["mouse__gene_name"].values
)
print(gene_mask.sum())
ce_mouse_blood_one2one = ce_mouse_blood[:, gene_mask]
ce_mouse_blood_one2one

13632


View of AnnData object with n_obs × n_vars = 15989 × 13632 
    obs: 'n_genes', 'n_counts', 'channel', 'tissue', 'species', 'free_annotation', 'narrow_group', 'broad_group', 'compartment_group', 'individual', 'species_latin'

## Use human gene names

In [19]:
ce_mouse_blood_one2one_new_var = ce_mouse_blood_one2one.var.merge(gene_orthologies_one2one_all_three, left_index=True, right_on='mouse__gene_name')
print(ce_mouse_blood_one2one_new_var.shape)
ce_mouse_blood_one2one_new_var.head()

(13632, 3)


,mouse_lemur__gene_name,human__gene_name,mouse__gene_name
9515,XKR4,XKR4,Xkr4
13307,SOX17,SOX17,Sox17
12415,MRPL15,MRPL15,Mrpl15
14509,LYPLA1,LYPLA1,Lypla1
10122,TCEA1,TCEA1,Tcea1


### Make new object with human gene names

In [20]:
ce_mouse_blood_one2one_human_genes = ce_mouse_blood_one2one.copy()
ce_mouse_blood_one2one_human_genes

AnnData object with n_obs × n_vars = 15989 × 13632 
    obs: 'n_genes', 'n_counts', 'channel', 'tissue', 'species', 'free_annotation', 'narrow_group', 'broad_group', 'compartment_group', 'individual', 'species_latin'

In [21]:
ce_mouse_blood_one2one_human_genes.var = ce_mouse_blood_one2one_new_var.set_index('human__gene_name')
ce_mouse_blood_one2one_human_genes.var.head()

,mouse_lemur__gene_name,mouse__gene_name
human__gene_name,,
XKR4,XKR4,Xkr4
SOX17,SOX17,Sox17
MRPL15,MRPL15,Mrpl15
LYPLA1,LYPLA1,Lypla1
TCEA1,TCEA1,Tcea1


## Write mouse all tissues, 1:1 orthologs, human gene names to file

### Write h5ad

In [22]:
%%time

for d in outdirs:
    h5ad = f"{d}/blood__10x__ce_mouse__one2one_orthologs.h5ad"
    %time ce_mouse_blood_one2one_human_genes.write(h5ad)

CPU times: user 261 ms, sys: 670 ms, total: 931 ms
Wall time: 29.4 s
CPU times: user 216 ms, sys: 657 ms, total: 873 ms
Wall time: 1.63 s
CPU times: user 479 ms, sys: 1.33 s, total: 1.81 s
Wall time: 31 s


In [58]:
ce_mouse_blood_one2one_human_genes

AnnData object with n_obs × n_vars = 15989 × 13632 
    obs: 'n_genes', 'n_counts', 'channel', 'tissue', 'species', 'free_annotation', 'narrow_group', 'broad_group', 'compartment_group', 'individual', 'species_latin'
    var: 'mouse_lemur__gene_name', 'mouse__gene_name'

In [59]:
ce_mouse_blood_one2one_human_genes.obs['narrow_group'].value_counts()

nan                                         6452
neutrophil                                  2685
B cell                                      1876
monocyte                                    1460
doublet                                     1315
CD4+ T cell                                  578
natural killer cell                          540
CD8+ T cell                                  477
macrophage                                   340
basophil                                      66
conventional dendritic cell                   39
plasmacytoid dendritic cell                   38
proliferating natural killer cell_T cell      37
dendritic cell                                20
CXCR6+ T cell                                 19
platelet                                      17
proliferating B cell_T cell                   16
TMEM176+ T cell                                9
erythroid lineage cell                         5
Name: narrow_group, dtype: int64

# Mouse data (Tabula Muris Senis), after unifying celltypes

## Load into Scanpy

In [23]:
%%time

h5ad = os.path.join(
    "/home/olga/data_lg/data_sm_copy/czb-tabula-muris-senis/Data-objects/",
    "tabula-muris-senis-droplet-official-raw-obj--no-duplicate-barcodes-per-seq-run--minimal-obs-unified-celltypes.h5ad",
)
mouse = scanpy.read_h5ad(h5ad)
mouse

CPU times: user 1.19 s, sys: 2.13 s, total: 3.32 s
Wall time: 12.8 s


AnnData object with n_obs × n_vars = 238915 × 20138 
    obs: 'age', 'cell_id', 'cell_ontology_class', 'cell_ontology_id', 'free_annotation', 'n_genes', 'sex', 'subtissue', 'tissue', 'species', 'species_latin', 'channel', 'sequencing_run', 'cell_barcode', 'n_counts', 'individual', 'narrow_group', 'broad_group', 'compartment_group'
    var: 'n_cells'

In [24]:
mouse.obs.narrow_group.value_counts()

nan                                         142270
B cell                                       29976
mesenchymal cell                             12712
T cell                                       11152
endothelial cell                              9668
monocyte                                      7926
macrophage                                    7005
proliferating monocyte                        2393
skeletal muscle satellite stem cell           2018
capillary cell                                1743
pericyte cell_smooth muscle cell              1554
alveolar fibroblast                           1499
alveolar macrophage                           1217
natural killer cell                           1141
interstitial macrophage                       1115
CD8+ T cell                                    854
neutrophil                                     543
CD4+ T cell                                    534
adventitial fibroblast                         517
capillary aerocyte cell        

In [25]:
mouse.obs.head()

,age,cell_id,cell_ontology_class,cell_ontology_id,free_annotation,n_genes,sex,subtissue,tissue,species,species_latin,channel,sequencing_run,cell_barcode,n_counts,individual,narrow_group,broad_group,compartment_group
index,,,,,,,,,,,,,,,,,,,
AAACCTGCAGTAAGCG-1-0-0-0,24m,MACA_24m_M_TONGUE_60_AAACCTGCAGTAAGCG,keratinocyte,NA,suprabasal,3481.0,male,nan,Tongue,Mouse,Mus musculus,MACA_24m_M_TONGUE_60,171103_A00111_0082_BH523JDMXX,AAACCTGCAGTAAGCG,1,mouse_18_24-M-60,nan,nan,nan
AAACCTGTCATTATCC-1-0-0-0,24m,MACA_24m_M_TONGUE_60_AAACCTGTCATTATCC,keratinocyte,NA,suprabasal,2599.0,male,nan,Tongue,Mouse,Mus musculus,MACA_24m_M_TONGUE_60,171103_A00111_0082_BH523JDMXX,AAACCTGTCATTATCC,1,mouse_18_24-M-60,nan,nan,nan
AAACGGGGTACAGTGG-1-0-0-0,24m,MACA_24m_M_TONGUE_60_AAACGGGGTACAGTGG,keratinocyte,NA,suprabasal differentiating,3468.0,male,nan,Tongue,Mouse,Mus musculus,MACA_24m_M_TONGUE_60,171103_A00111_0082_BH523JDMXX,AAACGGGGTACAGTGG,1,mouse_18_24-M-60,nan,nan,nan
AAACGGGGTCTTCTCG-1-0-0-0,24m,MACA_24m_M_TONGUE_60_AAACGGGGTCTTCTCG,keratinocyte,NA,suprabasal differentiating,3189.0,male,nan,Tongue,Mouse,Mus musculus,MACA_24m_M_TONGUE_60,171103_A00111_0082_BH523JDMXX,AAACGGGGTCTTCTCG,1,mouse_18_24-M-60,nan,nan,nan
AAAGATGAGCTATGCT-1-0-0-0,24m,MACA_24m_M_TONGUE_60_AAAGATGAGCTATGCT,keratinocyte,NA,suprabasal,3419.0,male,nan,Tongue,Mouse,Mus musculus,MACA_24m_M_TONGUE_60,171103_A00111_0082_BH523JDMXX,AAAGATGAGCTATGCT,1,mouse_18_24-M-60,nan,nan,nan


## Subset on 1:1 orthologs

In [26]:
gene_mask = mouse.var.index.isin(
    gene_orthologies_one2one_all_three["mouse__gene_name"].values
)
print(gene_mask.sum())
mouse_one2one = mouse[:, gene_mask]
mouse_one2one

13694


View of AnnData object with n_obs × n_vars = 238915 × 13694 
    obs: 'age', 'cell_id', 'cell_ontology_class', 'cell_ontology_id', 'free_annotation', 'n_genes', 'sex', 'subtissue', 'tissue', 'species', 'species_latin', 'channel', 'sequencing_run', 'cell_barcode', 'n_counts', 'individual', 'narrow_group', 'broad_group', 'compartment_group'
    var: 'n_cells'

In [27]:
gene_orthologies.head()

,NCBI_lemur_geneID,NCBI_lemur_geneName,NCBI_lemur_geneDescription,NCBI_lemur_geneType,Ensembl_human_homology,Ensembl_mouse_homology,NCBI_human_homology,NCBI_mouse_homology,any_mouse_homology,any_human_homology,NCBI_human_geneID_1,NCBI_human_geneName_1,NCBI_mouse_geneID_1,NCBI_mouse_geneName_1,Ensembl_lemur_geneID_1,Ensembl_lemur_geneID_2,Ensembl_lemur_geneName_1,Ensembl_lemur_geneName_2,Ensembl_human_geneID_1,Ensembl_human_geneID_2,Ensembl_human_geneName_1,Ensembl_human_geneName_2,Ensembl_mouse_geneID_1,Ensembl_mouse_geneID_2,Ensembl_mouse_geneName_1,Ensembl_mouse_geneName_2,Ensembl_human_geneID_AGG,Ensembl_human_geneName_AGG,Ensembl_mouse_geneID_AGG,Ensembl_mouse_geneName_AGG,NCBI_human_geneID_AGG,NCBI_human_geneName_AGG,NCBI_mouse_geneID_AGG,NCBI_mouse_geneName_AGG
1,105854774,PDE6D,phosphodiesterase 6D,protein-coding,ortholog_one2one,ortholog_one2one,ortholog,ortholog,yes,yes,5147.0,PDE6D,18582.0,Pde6d,ENSMICG00000032098,NaN,PDE6D,NaN,ENSG00000156973,NaN,PDE6D,NaN,ENSMUSG00000026239,NaN,Pde6d,NaN,ENSG00000156973,PDE6D,ENSMUSG00000026239,Pde6d,5147.0,PDE6D,18582.0,Pde6d
2,105854783,NUDT12,nudix hydrolase 12,protein-coding,ortholog_one2one,ortholog_one2one,ortholog,ortholog,yes,yes,83594.0,NUDT12,67993.0,Nudt12,ENSMICG00000036782,NaN,NUDT12,NaN,ENSG00000112874,NaN,NUDT12,NaN,ENSMUSG00000024228,NaN,Nudt12,NaN,ENSG00000112874,NUDT12,ENSMUSG00000024228,Nudt12,83594.0,NUDT12,67993.0,Nudt12
3,105854790,PI15,peptidase inhibitor 15,protein-coding,ortholog_one2one,ortholog_one2one,ortholog,ortholog,yes,yes,51050.0,PI15,94227.0,Pi15,ENSMICG00000003386,NaN,PI15,NaN,ENSG00000137558,NaN,PI15,NaN,ENSMUSG00000067780,NaN,Pi15,NaN,ENSG00000137558,PI15,ENSMUSG00000067780,Pi15,51050.0,PI15,94227.0,Pi15
4,105854791,GDAP1,ganglioside induced differentiation associated...,protein-coding,ortholog_one2one,ortholog_one2one,ortholog,ortholog,yes,yes,54332.0,GDAP1,14545.0,Gdap1,ENSMICG00000026766,NaN,GDAP1,NaN,ENSG00000104381,NaN,GDAP1,NaN,ENSMUSG00000025777,NaN,Gdap1,NaN,ENSG00000104381,GDAP1,ENSMUSG00000025777,Gdap1,54332.0,GDAP1,14545.0,Gdap1
5,105854792,JPH1,junctophilin 1,protein-coding,ortholog_one2one,ortholog_one2one,ortholog,ortholog,yes,yes,56704.0,JPH1,57339.0,Jph1,ENSMICG00000000667,NaN,JPH1,NaN,ENSG00000104369,NaN,JPH1,NaN,ENSMUSG00000042686,NaN,Jph1,NaN,ENSG00000104369,JPH1,ENSMUSG00000042686,Jph1,56704.0,JPH1,57339.0,Jph1


In [28]:
mouse.var.index.isin(gene_orthologies['NCBI_mouse_geneName_AGG'].values).sum()

13820

In [29]:
mouse.var.index.isin(gene_orthologies['Ensembl_mouse_geneName_AGG'].values).sum()

13814

In [30]:
mouse.var.index.isin(gene_orthologies['NCBI_mouse_geneName_1'].values).sum()

13441

In [31]:
mouse_one2one.var.head()

,n_cells
index,
Xkr4,147
Sox17,27289
Mrpl15,95312
Lypla1,86096
Tcea1,120238


## Use human gene names

In [32]:
mouse_one2one_new_var = mouse_one2one.var.merge(gene_orthologies_one2one_all_three, left_index=True, right_on='mouse__gene_name')
print(mouse_one2one_new_var.shape)
mouse_one2one_new_var.head()

(13694, 4)


,n_cells,mouse_lemur__gene_name,human__gene_name,mouse__gene_name
9515,147,XKR4,XKR4,Xkr4
13307,27289,SOX17,SOX17,Sox17
12415,95312,MRPL15,MRPL15,Mrpl15
14509,86096,LYPLA1,LYPLA1,Lypla1
10122,120238,TCEA1,TCEA1,Tcea1


### Make new object with human gene names

In [33]:
mouse_one2one_human_genes = mouse_one2one.copy()
mouse_one2one_human_genes

AnnData object with n_obs × n_vars = 238915 × 13694 
    obs: 'age', 'cell_id', 'cell_ontology_class', 'cell_ontology_id', 'free_annotation', 'n_genes', 'sex', 'subtissue', 'tissue', 'species', 'species_latin', 'channel', 'sequencing_run', 'cell_barcode', 'n_counts', 'individual', 'narrow_group', 'broad_group', 'compartment_group'
    var: 'n_cells'

In [34]:
mouse_one2one_human_genes.var = mouse_one2one_new_var.set_index('human__gene_name')
mouse_one2one_human_genes.var.head()

,n_cells,mouse_lemur__gene_name,mouse__gene_name
human__gene_name,,,
XKR4,147,XKR4,Xkr4
SOX17,27289,SOX17,Sox17
MRPL15,95312,MRPL15,Mrpl15
LYPLA1,86096,LYPLA1,Lypla1
TCEA1,120238,TCEA1,Tcea1


## Write mouse all tissues, 1:1 orthologs, human gene names to file

### Write h5ad

In [56]:
%%time

for d in outdirs:
    h5ad = f"{d}/alltissues__10x__tabula-muris-senis__one2one_orthologs.h5ad"
    %time mouse_one2one_human_genes.write(h5ad)

CPU times: user 5.08 s, sys: 3.08 s, total: 8.16 s
Wall time: 1min 20s
CPU times: user 4.61 s, sys: 1.52 s, total: 6.13 s
Wall time: 8.62 s
CPU times: user 9.7 s, sys: 4.61 s, total: 14.3 s
Wall time: 1min 28s


### Write CSVs

In [57]:
for d in outdirs:
    mouse_one2one.write_csvs(f'{d}/alltissues__10x__tabula-muris-senis__one2one_orthologs__csvs/')

writing .csv files to /home/olga/googledrive/TabulaMicrocebus/data/cross-species/alltissues__10x__tabula-muris-senis__one2one_orthologs__csvs
writing .csv files to /home/olga/data_lg/data_sm_copy/tabula-microcebus/data-objects/cross-species/alltissues__10x__tabula-muris-senis__one2one_orthologs__csvs


# Mouse Lemur raw counts

Created with this pull request: https://github.com/czbiohub/tabula-microcebus/pull/15

## Read all raw count data

In [37]:
%%time

# h5ad = f'{lemur_folder}/tabula-microcebus--10x--counts--min-51-genes--min-101-counts--trnas-summed--no-duplicate-barcodes--pairs-disambiguated--no-na.h5ad'
h5ad = os.path.join(
    "/home/olga/data_lg/data_sm_copy/tabula-microcebus/data-objects/ss2-10x-combined",
    "LCA_10x_update_minimal_obs.h5ad",
)
lemur = scanpy.read(h5ad, cache=True)
lemur

CPU times: user 1.04 s, sys: 1.79 s, total: 2.83 s
Wall time: 9.64 s


AnnData object with n_obs × n_vars = 231752 × 31509 
    obs: 'n_counts', 'n_genes', 'cell_id_v1', 'cell_id_v2', 'cell_barcode', 'sequencing_run', 'channel', 'tissue', 'subtissue', 'cell_ontology_class', 'free_annotation', 'species', 'species_latin', 'sex', 'age', 'individual', 'narrow_group', 'broad_group', 'compartment_group'
    var: 'name', 'highly_variable'
    uns: 'compartment_colors', 'compartment_update_colors', 'individual_colors', 'method_colors', 'tissue_colors'
    obsm: 'X_pca', 'X_umap'

## Prepare for concatenation and 1:1 orthologs

### Filter for only 1:1 orthologs

In [38]:
gene_mask = lemur.var.index.isin(gene_orthologies_one2one_all_three['mouse_lemur__gene_name'].values)
print(gene_mask.sum())
lemur_one2one = lemur[:, gene_mask]
lemur_one2one

15102


View of AnnData object with n_obs × n_vars = 231752 × 15102 
    obs: 'n_counts', 'n_genes', 'cell_id_v1', 'cell_id_v2', 'cell_barcode', 'sequencing_run', 'channel', 'tissue', 'subtissue', 'cell_ontology_class', 'free_annotation', 'species', 'species_latin', 'sex', 'age', 'individual', 'narrow_group', 'broad_group', 'compartment_group'
    var: 'name', 'highly_variable'
    uns: 'compartment_colors', 'compartment_update_colors', 'individual_colors', 'method_colors', 'tissue_colors'
    obsm: 'X_pca', 'X_umap'

In [39]:
lemur.var.head()

,name,highly_variable
AU129-gt01,AU129-gt01,False
AU129-gr02,AU129-gr02,True
AU129-gt02,AU129-gt02,False
AU129-gr01,AU129-gr01,True
AU129-gt03,AU129-gt03,False


In [40]:
lemur_one2one.var.head()

,name,highly_variable
RBP2,RBP2,False
COPB2,COPB2,True
MRPS22,MRPS22,False
FOXL2,FOXL2,False
PIK3CB,PIK3CB,False


### Use human gene names

In [41]:
lemur_one2one_new_var = lemur_one2one.var.merge(gene_orthologies_one2one_all_three, left_index=True, right_on='mouse_lemur__gene_name')
print(lemur_one2one_new_var.shape)
lemur_one2one_new_var.head()

(15102, 5)


,name,highly_variable,mouse_lemur__gene_name,human__gene_name,mouse__gene_name
8372,RBP2,False,RBP2,RBP2,Rbp2
12644,COPB2,True,COPB2,COPB2,Copb2
6060,MRPS22,False,MRPS22,MRPS22,Mrps22
9771,FOXL2,False,FOXL2,FOXL2,Foxl2
15015,PIK3CB,False,PIK3CB,PIK3CB,Pik3cb


In [42]:
lemur_one2one.var = lemur_one2one_new_var.set_index('human__gene_name')
lemur_one2one.var.head()

,name,highly_variable,mouse_lemur__gene_name,mouse__gene_name
human__gene_name,,,,
RBP2,RBP2,False,RBP2,Rbp2
COPB2,COPB2,True,COPB2,Copb2
MRPS22,MRPS22,False,MRPS22,Mrps22
FOXL2,FOXL2,False,FOXL2,Foxl2
PIK3CB,PIK3CB,False,PIK3CB,Pik3cb


## Write whole object to file

In [43]:
for d in outdirs:
    h5ad = f"{d}/alltissues__10x__lemur__one2one_orthologs.h5ad"
    %time lemur_one2one.write(h5ad)

#     lemur_one2one.write_csvs(f'{d}/alltissues__10x__lemur__one2one_orthologs__csvs/')

CPU times: user 6.93 s, sys: 3.84 s, total: 10.8 s
Wall time: 1min 58s
CPU times: user 6.17 s, sys: 1.62 s, total: 7.79 s
Wall time: 9.75 s


# Human lung Cell atlas data

## Read preprocessed Human h5ad

In [45]:
human_folder = '/home/olga/data_lg/data_sm_copy/tabula-microcebus/data-objects/human-lung-cell-atlas--from-kyle'
! ls -lha $human_folder

total 8.5G
drwxr-xr-x 2 olga czb 4.0K Feb 15 10:35 .
drwxrwxr-x 8 olga czb 4.0K Feb 15 10:35 ..
-rw-r--r-- 1 olga czb 8.5G Feb 15 10:35 droplet_normal_lung_blood_P1-3__no_duplicate_barcodes__minimal_obs.h5ad


In [46]:
%%time

h5ad = f"{human_folder}/droplet_normal_lung_blood_P1-3__no_duplicate_barcodes__minimal_obs.h5ad"
human = scanpy.read(h5ad, cache=True)
human

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


CPU times: user 765 ms, sys: 8.07 s, total: 8.84 s
Wall time: 15.1 s


AnnData object with n_obs × n_vars = 85623 × 26485 
    obs: 'n_genes', 'n_counts', 'channel', 'free_annotation', 'cell_ontology_class', 'species', 'species_latin', 'sequencing_run', 'cell_barcode', 'individual', 'age', 'sex', 'narrow_group', 'broad_group', 'compartment_group', 'tissue'

## Subset to 1:1 orthologs

In [47]:
gene_mask = human.var.index.isin(gene_orthologies_one2one_all_three['human__gene_name'].values)
print(gene_mask.sum())

# Need to copy so it's not an ArrayView object
human_one2one = human[:, gene_mask].copy()
human_one2one

14854


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


AnnData object with n_obs × n_vars = 85623 × 14854 
    obs: 'n_genes', 'n_counts', 'channel', 'free_annotation', 'cell_ontology_class', 'species', 'species_latin', 'sequencing_run', 'cell_barcode', 'individual', 'age', 'sex', 'narrow_group', 'broad_group', 'compartment_group', 'tissue'

In [48]:
human_one2one.var = gene_orthologies_one2one_all_three.set_index('human__gene_name').loc[human_one2one.var.index]
human_one2one.var.head()

,mouse_lemur__gene_name,mouse__gene_name
index,,
SAMD11,SAMD11,Samd11
NOC2L,NOC2L,Noc2l
KLHL17,KLHL17,Klhl17
PLEKHN1,PLEKHN1,Plekhn1
PERM1,PERM1,Perm1


### Check on "LOC" genes in mouse lemur

In [49]:
gene_orthologies_one2one_all_three.loc[gene_orthologies_one2one_all_three.mouse_lemur__gene_name.str.startswith("LOC")]

,mouse_lemur__gene_name,human__gene_name,mouse__gene_name
60,LOC105858168,HSPA1L,Hspa1l
75,LOC105869751,HLA-DQB1,H2-Ab1
118,LOC105877660,SLC35E2B,Slc35e2
152,LOC105873725,KIAA1958,E130308A19Rik
159,LOC105878821,H3-4,H3f4
...,...,...,...
15235,LOC105864454,OR6K3,Olfr424
15323,LOC105857443,TRMT9B,Trmt9b
15342,LOC105883627,FAHD2A,Fahd2a
15349,LOC105854855,COX5A,Cox5a


## Write human to file

In [50]:
for d in outdirs:

    h5ad = f"{d}/hlca_lung_blood_10x__human__one2one_orthologs.h5ad"
    human_one2one.write(h5ad)

#     human_one2one.write_csvs(f'{d}/hlca_lung_blood__10x__human__one2one_orthologs__csvs/')


In [51]:
! ls -lha $h5ad

-rw-r--r-- 1 olga czb 4.8G Feb 15 11:15 /home/olga/data_lg/data_sm_copy/tabula-microcebus/data-objects/cross-species/hlca_lung_blood_10x__human__one2one_orthologs.h5ad


# Human: Tabula Sapiens

In [50]:
h5ad = os.path.join('/home/olga/data_lg/data_sm_copy/tabula-sapiens/data-objects/Pilot1_Pilot2_decontX_Oct2020__minimal_no_pilot2_diaphragm.h5ad')
sapiens = scanpy.read(h5ad, cache=True)
sapiens

AnnData object with n_obs × n_vars = 132884 × 58870 
    obs: 'individual', 'tissue', 'subtissue', 'cell_barcode', 'channel', 'sex', 'n_counts', 'n_genes', 'cell_ontology_class', 'age', 'species', 'species_latin', 'narrow_group', 'broad_group', 'compartment_group'
    var: 'ensembl_id', 'len'
    uns: 'Propagated.Annotationcollapsed_colors', '_scvi', 'annotation_oct2020_colors', 'cell_ontology_type_colors', 'compartment_colors', 'compartment_oct2020_colors', 'donor_colors', 'leiden', 'method_colors', 'neighbors', 'tissue_colors', 'umap'
    obsm: 'X_scvi', 'X_scvi_decontX', 'X_scvi_raw', 'X_umap', 'X_umap_decontX', 'X_umap_raw'
    layers: 'raw_counts'
    obsp: 'connectivities', 'distances'

## Subset to 1:1 orthologs

In [51]:
gene_mask = sapiens.var.index.isin(gene_orthologies_one2one_all_three['human__gene_name'].values)
print(gene_mask.sum())

# Need to copy so it's not an ArrayView object
sapiens_one2one = sapiens[:, gene_mask].copy()
sapiens_one2one

15318


AnnData object with n_obs × n_vars = 132884 × 15318 
    obs: 'individual', 'tissue', 'subtissue', 'cell_barcode', 'channel', 'sex', 'n_counts', 'n_genes', 'cell_ontology_class', 'age', 'species', 'species_latin', 'narrow_group', 'broad_group', 'compartment_group'
    var: 'ensembl_id', 'len'
    uns: 'Propagated.Annotationcollapsed_colors', '_scvi', 'annotation_oct2020_colors', 'cell_ontology_type_colors', 'compartment_colors', 'compartment_oct2020_colors', 'donor_colors', 'leiden', 'method_colors', 'neighbors', 'tissue_colors', 'umap'
    obsm: 'X_scvi', 'X_scvi_decontX', 'X_scvi_raw', 'X_umap', 'X_umap_decontX', 'X_umap_raw'
    layers: 'raw_counts'
    obsp: 'connectivities', 'distances'

In [52]:
sapiens_one2one.var = gene_orthologies_one2one_all_three.set_index('human__gene_name').loc[sapiens_one2one.var.index]
sapiens_one2one.var.head()

,mouse_lemur__gene_name,mouse__gene_name
SAMD11,SAMD11,Samd11
NOC2L,NOC2L,Noc2l
KLHL17,KLHL17,Klhl17
PLEKHN1,PLEKHN1,Plekhn1
PERM1,PERM1,Perm1


## Write sapiens subset to file

In [53]:
sapiens_h5ad = os.path.join(outdir_local, 'tabula_sapiens__10x__human__one2one_orthologs.h5ad')
sapiens.write(sapiens_h5ad)

In [55]:
! ls -lha $sapiens_h5ad

-rw-r--r-- 1 olga czb 5.6G Feb 11 12:59 /home/olga/data_lg/data_sm_copy/tabula-microcebus/data-objects/cross-species/tabula_sapiens__10x__human__one2one_orthologs.h5ad
